UCLA Extension - Introduction to Data Science
<br>COM SCI X 450.1
<br>Author: Nathan Strong
<br>Instructor: Ali El-Annan
<br>Date: May 2020

# Part 1: Comparing the databases Microsoft SQL Server (SQL) and Google Firestore (NoSQL)
# Part 2: Comparing algorithms Naive Bayes (supervised) and K-Means Clustering (unsupervised)

### Forward
This essay will compare and contrast two popular databases as well as two popular algorithms.  None of the math for the algorithms is my own (obviously), but the code to generate the following examples is all mine -- It can be downloaded from my GitHub repo here: https://github.com/NaughtyPhoton/ucla-data-science

### The Study -- What is the Most Popular Programming Language as Decided by Twitter
As a means of displaying the databases and algorithms in action, I will be doing a "study" of preferred programming languages as decided by Twitter.  This study is merely a vehicle to depict the technology in question. 
<br>The programming languages I would like to examine are the 5 "Most Loved" programming languages of 2019 as voted by the Stack Overflow Community in their Annual Developer's Survey: https://insights.stackoverflow.com/survey/2019

In [9]:
%load_ext autoreload
%autoreload 2

from typing import Tuple

# In order of most loved -> least loved
SO_MOST_LOVED: Tuple[str] = ('Rust', 'Python', 'TypeScript', 'Kotlin', 'WebAssembly')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Collecting Tweets with Twython
Using the Python library Twython I can search for Tweets. Ideally, I would collect all the tweets mentioning our most loved languages within the past year, however, the Twitter API won't allow you to return that many results. Instead, I'll use the Twython.TwythonStreamer class to collect tweets while listening for a period of time.  I've already extended the TwythonStreamer class in another file to compress this document.

In [7]:
from typing import List
from collections import defaultdict
import time
from custom_streamer import CustomStreamer
from tweet import Tweet

MINUTES_TO_LISTEN = 1

language_tweets_dict: dict = {}
for language in SO_MOST_LOVED: 
    twitterStreamer = CustomStreamer(minutes_to_listen=MINUTES_TO_LISTEN)
    twitterStreamer.statuses.filter(track=language)
    language_tweets_dict[language] = twitterStreamer.tweets
    
    print(f'{language} recieved {len(language_tweets_dict[language])} tweets during {MINUTES_TO_LISTEN} minute(s).')

Rust recieved 43 tweets during 1 minute(s).
420 b'Exceeded connection limit for user\r\n'
Python recieved 43 tweets during 1 minute(s).
420 b'Exceeded connection limit for user\r\n'
TypeScript recieved 43 tweets during 1 minute(s).
420 b'Exceeded connection limit for user\r\n'
Kotlin recieved 43 tweets during 1 minute(s).


ConnectionError: HTTPSConnectionPool(host='stream.twitter.com', port=443): Read timed out.

In [7]:
print(language_tweets_dict['Rust'][-1])

Tweet(id_str='1266209998566092801', text='Write multithreaded web apps in Rust!\n\nPublished a suite of new crates that facilitates mutithreading in Rust and W… https://t.co/AmEm9PECOp', date=datetime.datetime(2020, 5, 29, 3, 29, 23, tzinfo=tzutc()), user_handle='w3reality', user_description='Elevating the Web to VR\nedge computing | dataviz | oss | software engineering', hashtags=[])


# Part 1 - Databases
It's a good idea to store my collection of tweets in a database, as the Twitter API only allows for a limited amount of API calls. This way I can also add to my Tweet collection by running this script on different days, thus growing my sample size, and when I feel my collection is sufficient, I can stop querying for new Tweets.
## Part 1.1 - SQL Databases
#### What is an SQL Database?
SQL stands for Structured Query Language; the language used to work with a relational database.  A relational database is one which finds data based on its relationship to other data.  The SQL language is (pretty much) the same between different DBMS (Database Management Systems).
#### Microsoft SQL Server
I chose to use the Microsoft SQL Server for this project because I have never used it before and people seem to really like it. It uses it's own flavor of SQL called "transact-sql" (t-sql).  It has an integrated envioronment called Microsoft SQL Manager which provides GUI tools for managing databases.  The Microsoft SQL Server can also live on a Microsoft Azure instance if the database is designed to work on the cloud.  It has a ton of other features, but notably (for data science) it has a service called SQL Server Analysis Services (SSAS) that provides tools for Machine Learning and Data Analysis (I won't be using these).

First, I'll define my table, columns, and data types in the GUI app, but this can also be done directly through SQL as well.

<img src="images/SQL_db.jpg" />

Since SQL is a relational database, we'll need a "foreign key, in my table which is a key that matches the "primary key" of another table so the data can be easily combined. In my case, I am only going to create this one table, but if I were to continue with this experiment, I would make the "language" column a foreign key so that I could make a seperate collection of languages and easily qeuery all the tweets for each language.

<img src="images/SQL_table_keys.png" />

#### Using Python with Microsoft SQL Server - pyodbc
The easiest way to update my SQL server is with the Python library pyodbc. pyodc is an open-source Python library which provides CRUD (Create, Read, Update, Delete) methods for ODBC (Open Database Connectivity) enabled DBMS.

In [2]:
import pyodbc

# Connect to SQL Server
connect_string = 'Driver={SQL Server Native Client RDA 11.0}; \
                  Server=DESKTOP-OOMFKNN; \
                  Database=programming_tweets; \
                  Trusted_Connection=yes;'

with pyodbc.connect(connect_string) as conn:
    cursor = conn.cursor()
    
    # Define the columns and values to be updated    
    insert_query = """INSERT INTO tweets (language, id_str, text, date, user_handle, user_description)
                      VALUES (?, ?, ?, ?, ?, ?)"""
    
    # Loop through all the tweets
    for language, tweets in language_tweets_dict.items():
        for _tweet in tweets:
            # Define the new values to insert             
            values = (language, _tweet.id_str, _tweet.text, _tweet.date, _tweet.user_handle, _tweet.user_description)
            
            # Insert the new values into the database
            cursor.execute(insert_query, values)
    
    # Commit the inserts     
    conn.commit()

Now that the database has been updated, we can read the information inside of it via a 'SELECT' query. Below, I am printing the last 5 tweets in the collection. As you can see, we'll need to do some filtering to determine which tweets are actually about programming.

In [6]:
with pyodbc.connect(connect_string) as conn:
    cursor = conn.cursor()
    
    # Grab all the rows in the tweets table
    cursor.execute('SELECT user_handle, text FROM tweets')
    
    [print(f'user: {x[0]}\ntweet: {x[1]}\n') for x in list(cursor)[:5]]

user: Tall_Individual
tweet: @RunningEagle11 @SteveRustad1 Yea he'll definitely get the rust belt,Florida,Michigan,and Pennsylvania....suuurrree… https://t.co/5p17g2U5Zt

user: enderton_justin
tweet: Renewable energy can generate billions of dollars in health benefits, study finds #RenewableEnergy #health via… https://t.co/9JoBDSHFn2

user: GoatedPS
tweet: @FussyAF @xollost @Raintastical Obviously my son miss his pops Xolo so we not gonna talk about that game w rust hurdddddd it

user: AdamRust9
tweet: RT @NCRC: At a time when underserved neighborhoods face so many challenges, we’re expanding our team to both understand and drive innovatio…

user: anglehernandez
tweet: RT @realpython: Fil: A New Python Memory Profiler for Data Scientists and Scientists #python https://t.co/FFy9ErcNS4



## Part 1.2 - NoSQL Databases
### What is a NoSQL Database?
A NoSQL database is not as strictly defined as a SQL database.  It is really any database that is not a SQL database. The key difference is that a SQL database finds data 'relationally' by performing qeueries that group shared columns together; where-as a NoSQL database generally needs a path or an id that points directly to the file in question. Data can still be stored relationally, in that a piece of data can hold an id pointing to another piece of data, but this type of query would take two seperate commands to get to that second piece of data.

Where a SQL database enforces strict typing and defining columns ahead of time, a NoSQL database is "schema-less" meaning that the type of data isn't enforced when it is stored. Furthermore, there is no consistent language used in a NoSQL database, such as the SQL language or CRUD operation workflow, rather, access to data is usually governed by a RESTful API (REpresentational State Transfer), a style for building systems that ensures consistent results. 

A schema-less interface is convenient for a project like mine, where I might discover a new field that I need while working with the data, I don't need to modify every piece of data in the database to add the new field.
<img src="images/NoSQL_documents.png" />

Not having SQL means that we have to get creative at times with how we store and retrieve data. For example, say I want to store all my tweets as well as the users who tweeted them.  I probably want to have a seperate collection of users, and because we can no longer use SQL joins to collect users and tweets in one queury, it's better to make duplicate data in the two collections. This is because it is generally better to optimize reading data than writing it.
<img src="images/NoSQL_duplicate_data.png" />

One of the biggest benefits of a NoSQL database is the way it is able to scale. When we have a lot of users on a SQL database, we need to scale vertically, meaning, putting our database on bigger machines. With a NoSQL database, the DBMS can spread the data across multiple servers behind the scenes, providing faster data access for users; this is known as scaling horizontally.

### Google Firestore Database
Google's Firestore, is a NoSQL database in the cloud. Firestore is a "document" database, meaning that each datapoint is not a table, rather a document in a collection. Additionally, each document may contain more collections within. 

Perhaps the coolest part of Firestore is its realtime capabilities. I could make a website that uses JavaScript to display graphs depicting my study that "subscribes" to the data, and each time it is updated, my website will recieve the updates and immediately display the results without the need for another qeuery.

Firestore can be extended with "Cloud Functions" which run on the database at specified triggers such as new documents being added, I won't be exploring this right now.

Lets add our tweets to Firestore using the Python API.

In [4]:
from firebase import firebase_auth

# Get the db connection from firestore
db = firebase_auth.get_db()

# Get the tweets collection
tweets_collection = db.collection('tweets')

# Loop through all the tweets and add them to a batch, so we set them all at once
batch = db.batch()
for language, tweets in language_tweets_dict.items():
    for _tweet in tweets:
        document_id = _tweet.id_str
        data = _tweet.__dict__
        data['language'] = language
        batch.set(tweets_collection.document(document_id), data)

batch.commit()

[update_time {
   seconds: 1590821955
   nanos: 454145000
 },
 update_time {
   seconds: 1590821955
   nanos: 454145000
 },
 update_time {
   seconds: 1590821955
   nanos: 454145000
 },
 update_time {
   seconds: 1590821955
   nanos: 454145000
 },
 update_time {
   seconds: 1590821955
   nanos: 454145000
 },
 update_time {
   seconds: 1590821955
   nanos: 454145000
 },
 update_time {
   seconds: 1590821955
   nanos: 454145000
 },
 update_time {
   seconds: 1590821955
   nanos: 454145000
 },
 update_time {
   seconds: 1590821955
   nanos: 454145000
 },
 update_time {
   seconds: 1590821955
   nanos: 454145000
 },
 update_time {
   seconds: 1590821955
   nanos: 454145000
 },
 update_time {
   seconds: 1590821955
   nanos: 454145000
 },
 update_time {
   seconds: 1590821955
   nanos: 454145000
 },
 update_time {
   seconds: 1590821955
   nanos: 454145000
 },
 update_time {
   seconds: 1590821955
   nanos: 454145000
 },
 update_time {
   seconds: 1590821955
   nanos: 454145000
 },
 update_